In [21]:
import torch
import numpy as np
import torch.nn as nn

# Basics

In [5]:
torch.empty(2, 2).size()

torch.Size([2, 2])

In [11]:
torch.rand(1,1)

tensor([[0.1964]])

In [9]:
b = torch.tensor([1, 2])
b.add_(1)
b

tensor([2, 3])

In [16]:
# reshape
a = torch.empty(4, 3).fill_(5)
print(f'{a=}')
b = a.view(4*3)
print(f'{b=}')

a=tensor([[5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.]])
b=tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])


In [20]:
a_numpy_link = a.numpy()
a_numpy_link[:, 0] = 1
a

tensor([[1., 5., 5.],
        [1., 5., 5.],
        [1., 5., 5.],
        [1., 5., 5.]])

In [27]:
np_arr = np.array([1, 2, 3], dtype=np.int32)
arr_link = torch.from_numpy(np_arr)
arr_link[0] = 0
np_arr

array([0, 2, 3])

# Autograd

In [56]:
x = torch.tensor([1, 2, 3, 4, 5], dtype=float, requires_grad=True)
print(x + 1)
y = x.mean()
print(y)
y.backward()
print(x.grad)

tensor([2., 3., 4., 5., 6.], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(3., dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)


In [64]:
w = torch.ones(4, requires_grad=True)

for epoch in range(5):
    y = 3*w.sum()
    y.backward()

    print(f'epoch №{epoch}: {w.grad}')

epoch №0: tensor([3., 3., 3., 3.])
epoch №1: tensor([6., 6., 6., 6.])
epoch №2: tensor([9., 9., 9., 9.])
epoch №3: tensor([12., 12., 12., 12.])
epoch №4: tensor([15., 15., 15., 15.])


In [66]:
w = torch.ones(4, requires_grad=True)

for epoch in range(5):
    y = 3*w.sum()
    y.backward()

    print(f'epoch №{epoch}: {w.grad}')

    # correct
    w.grad.zero_()

epoch №0: tensor([3., 3., 3., 3.])
epoch №1: tensor([3., 3., 3., 3.])
epoch №2: tensor([3., 3., 3., 3.])
epoch №3: tensor([3., 3., 3., 3.])
epoch №4: tensor([3., 3., 3., 3.])


# Backpropagation

In [150]:
x = torch.tensor(1.)
y = torch.tensor(2.)

w = torch.tensor(1., requires_grad=True)

def forward(x):
    return w * x

def mse(y, y_pred):
    return torch.mean((y - y_pred)**2)

learning_rate = 0.4
for epoch in range(5):
    print(f'\nEPOCH №{epoch+1}')

    # forward pass: compute the loss
    y_hat = forward(x)
    loss = mse(y, y_hat)

    # backward pass: compute the gradients
    loss.backward()

    print(f'{loss=:.4f}, {w.grad=:.4f}, {w=:.4f}')

    with torch.no_grad():
        # update weights
        w -= learning_rate * w.grad
        w.grad.zero_()


EPOCH №1
loss=1.0000, w.grad=-2.0000, w=1.0000

EPOCH №2
loss=0.0400, w.grad=-0.4000, w=1.8000

EPOCH №3
loss=0.0016, w.grad=-0.0800, w=1.9600

EPOCH №4
loss=0.0001, w.grad=-0.0160, w=1.9920

EPOCH №5
loss=0.0000, w.grad=-0.0032, w=1.9984


# Training pipeline

In [268]:
X = torch.tensor([[1.]])
Y = torch.tensor([[2.]])

n_samples, n_features = X.shape

class LinearRegression(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.linear = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        return self.linear(x)
    
model = LinearRegression(n_samples, n_samples)
mse = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), 0.2)

for epoch in range(5):
    print(f'\nEPOCH №{epoch+1}')

    y_hat = model(X)
    loss = mse(Y, y_hat)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    w, b = model.parameters()
    print(f'{loss=:.4f}, {w[0][0]=:.4f}, {b[0]=:.4f}')


EPOCH №1
loss=6.0302, w[0][0]=0.9638, b[0]=0.5450

EPOCH №2
loss=0.2412, w[0][0]=1.1603, b[0]=0.7415

EPOCH №3
loss=0.0096, w[0][0]=1.1996, b[0]=0.7808

EPOCH №4
loss=0.0004, w[0][0]=1.2074, b[0]=0.7886

EPOCH №5
loss=0.0000, w[0][0]=1.2090, b[0]=0.7902


# Linear regression